In [ ]:
# Function to store data in PostgreSQL
def store_data_in_postgres(artists_data, dbname, user, password, host, port):
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()
    
    # Create tables if they don't exist
    cur.execute("""
        CREATE TABLE IF NOT EXISTS artists (
            artist_id VARCHAR PRIMARY KEY,
            name VARCHAR,
            followers INT,
            popularity INT,
            genres TEXT[],
            images TEXT[]
        );
        
        CREATE TABLE IF NOT EXISTS tracks (
            track_id VARCHAR PRIMARY KEY,
            name VARCHAR,
            artist_id VARCHAR REFERENCES artists(artist_id),
            album_id VARCHAR,
            popularity INT,
            release_date DATE
        );
    """)
    
    # Insert artist data and commit before inserting tracks data to avoid ForeignKeyViolation
    for artist in artists_data['artists']:
        cur.execute("""
            INSERT INTO artists (artist_id, name, followers, popularity, genres, images)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (artist_id) DO NOTHING;
        """, (artist['id'], artist['name'], artist['followers']['total'], artist['popularity'], artist['genres'], [img['url'] for img in artist['images']]))
    
    conn.commit()
    
    # Insert tracks data
    for artist in artists_data['artists']:
        popular_songs = get_popular_songs(access_token, artist['id'])
        
        for song in popular_songs['tracks']:
            release_date = song['album']['release_date']
            try:
                release_date = datetime.strptime(release_date, '%Y-%m-%d').date()
            except ValueError:
                try:
                    release_date = datetime.strptime(release_date + '-01-01', '%Y-%m-%d').date()
                except ValueError:
                    release_date = None
            
            cur.execute("""
                INSERT INTO tracks (track_id, name, artist_id, album_id, popularity, release_date)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (track_id) DO NOTHING;
            """, (song['id'], song['name'], song['artists'][0]['id'], song['album']['id'], song['popularity'], release_date))
    
    conn.commit()
    cur.close()
    conn.close()

# Spotify API credentials (replace with your own)
client_id = 'Your client ID'
client_secret = 'Your client_secret ID'

# PostgreSQL database credentials (replace with your own)
dbname='Your Database name'
user='your user name'
password='your password'
host='host'
port='your port'

# Get access token
access_token = get_access_token(client_id, client_secret)

if access_token:
    # Get top 10 artists
    top_artists = get_top_artists(access_token)

    # Store data in PostgreSQL
    store_data_in_postgres(top_artists, dbname, user, password, host, port)

    print("Data has been successfully fetched and stored in PostgreSQL.")
else:
    print("Failed to obtain access token.")